In [30]:
%run ../00_default_options.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from robotehr.api.training import get_training_results
from robotehr.pipelines import training
from robotehr.models.cohort import Cohort, OnsetDataFrame
from robotehr.models.data import FeaturePipeline
from robotehr.pipelines.supporters.preprocessing import DataLoader

In [32]:
cohort = Cohort.load(id=1)

In [33]:
onset_dataframe = OnsetDataFrame.load(id=15)

In [34]:
feature_pipeline = FeaturePipeline.load(id=21)

In [35]:
import morpher.config
from morpher.jobs import *

In [36]:
regexes = [
    'any__Diagnosis__',
    #'any__Procedure__', 
    #'any__Drug__', 
    #'any__Material__', 
    #'any__Encounter__', 
    #'any__AlcoholUse__', 
    #'any__DrugUse__', 
    #'any__TobaccoUse__',
    'min__height__', 
    'min__weight__', 
    'min__vitalsign__', 
    #'min__measurement__', 
    'min__labvalue__', 
    'max__height__', 
    'max__weight__', 
    'max__vitalsign__', 
    #'max__measurement__', 
    'max__labvalue__'
]

agg_func_regex = ""
for r in regexes:
    agg_func_regex += r + "|"
agg_func_regex = agg_func_regex[:-1]

def prepare_data_function(data, target):
    import morpher
    from morpher.jobs import Impute, Scale
    from sklearn.preprocessing import OneHotEncoder
    from enum import Enum
    
    # remove unused features
    del data['medical_record_number']
    del data['mother_account_number']
    del data['date_of_birth']
    del data['month_of_birth']
    del data['patient_ethnic_group']
    del data['religion']
    del data['address_zip']
    del data['deceased_indicator']
    del data['marital_status_code']
    
    class RaceType(str, Enum):
        AFRICAN = 'African'
        AMERICAN_BLACK = 'Black or African-American'
        AMERICAN_NATIVE = 'Native American'
        ASIAN = 'Asian'
        ASIAN_PACIFIC = 'Asian Pacific'
        ASIAN_INDIAN = 'Asian Indian'
        ASIAN_CHINESE = 'Asian Chinese'
        HISPANIC = 'Hispanic or Latino'
        OTHER = 'Other'
        WHITE = 'White'
    
    RACE_MAPPING = {
        RaceType.AFRICAN: [
            'Cape Verdian',
            'Congolese',
            'Eritrean',
            'Ethiopian',
            'Gabonian',
            'Ghanaian',
            'Guinean',
            'Ivory Coastian',
            'Kenyan',
            'Liberian',
            'Madagascar',
            'Malian',
            'Nigerian',
            'Other: East African',
            'Other: North African',
            'Other: South African',
            'Other: West African',
            'Senegalese',
            'Sierra Leonean',
            'Somalian',
            'Sudanese',
            'Tanzanian',
            'Togolese',
            'Ugandan',
            'Zimbabwean'
        ],
        RaceType.AMERICAN_BLACK: [
            'African American (Black)',
            'African-American',
            'Black Or African-American',
            'Black or African - American',
        ],
        RaceType.AMERICAN_NATIVE: [
            'American (Indian/Alaskan)',
            'Native American'
        ],
        RaceType.ASIAN: [
            'Asian',
            'Bangladeshi',
            'Bhutanese',
            'Burmese',
            'Cambodian',
            'Hmong',
            'Indonesian',
            'Japanese',
            'Korean',
            'Laotian',
            'Malaysian',
            'Maldivian',
            'Nepalese',
            'Okinawan',
            'Pakistani',
            'Singaporean',
            'Taiwanese',
            'Thai',
            'Vietnamese',
            'Yapese'
        ],
        RaceType.ASIAN_PACIFIC: [
            'Asian (Pacific Islander)',
            'Carolinian',
            'Chamorro',
            'Chuukese',
            'Fijian',
            'Filipino',
            'Guamanian',
            'Guamanian Or Chamorro',
            'Guamanian or Chamorro',
            'Iwo Jiman',
            'Kiribati',
            'Kosraean',
            'Mariana Islander',
            'Marshallese',
            'Melanesian',
            'Micronesian',
            'Native Hawaiian',
            'New Hebrides',
            'Other Pacific Islander',
            'Pacific Islander',
            'Palauan',
            'Pohnpeian',
            'Polynesian',
            'Saipanese',
            'Samoan',
            'Papua New Guinean',
            'Tahitian',
            'Tokelauan',
            'Tongan'
        ],
        RaceType.ASIAN_INDIAN: [
            'Asian Indian',
            'Sri Lankan',
            'Sri lankan',
            'West Indian'
        ],
        RaceType.ASIAN_CHINESE: [
            'Chinese',
        ],
        RaceType.HISPANIC: [
            'Barbadian',
            'Dominica Islander',
            'Grenadian',
            'Haitian',
            'Hispanic/Latino',
            'Jamaican',
            'St Vincentian',
            'Trinidadian'
        ],
        RaceType.OTHER: [
            '',
            'Aa',
            'Ab',
            'Af',
            'Ag',
            'Ak',
            'Al',
            'Ap',
            'Ar',
            'Av',
            'Ay',
            'B',
            'B1',
            'B2',
            'B3',
            'B4',
            'B5',
            'B6',
            'B7',
            'B8',
            'B9',
            'Ba',
            'Bb',
            'Bc',
            'Bd',
            'Be',
            'Bf',
            'Bg',
            'Bh',
            'Bj',
            'Bk',
            'Bm',
            'Bn',
            'Bo',
            'Bp',
            'Bq',
            'Br',
            'Bs',
            'Bt',
            'Bu',
            'Bv',
            'Bw',
            'Bx',
            'By',
            'Bz',
            'I',
            'MSDW_NOT APPLICABLE',
            'MSDW_OTHER',
            'MSDW_UNKNOWN',
            'NOT AVAILABLE',
            'Non Hispanic',
            'O',
            'Other',
            'Pk',
            'Pl',
            'Pm',
            'Po',
            'Ps',
            'Pv',
            'U',
            'Unk',
            'Unknown',
            'W'
        ],
        RaceType.WHITE: [
            'Caucasian (White)',
            'White'
        ]
    }
    
    # value mapping
    data['race'] = (
        data.race.map({
            label: cat for cat, labels in RACE_MAPPING.items()
            for label in labels
        }).astype(pd.api.types.CategoricalDtype(RaceType))
    )
    
    # pre-encoding nan handling
    for column in ['gender', 'race']:
        enc = OneHotEncoder(sparse=False)
        transformed_data = enc.fit_transform(data[[column]])
        transformed_columns = pd.DataFrame(
            data=transformed_data, 
            columns=[f'{column}_{c}' for c in enc.categories_[0]]
        )
        data = data.join(transformed_columns)
        del data[column]
        
    label_encoded_data = data.copy()
    
    label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]] = label_encoded_data[label_encoded_data.columns[label_encoded_data.columns.str.contains('any')]].fillna(False)
    
    scaled_data, _ = Scale().execute(data=label_encoded_data, target=target)
    
    imputed_data, _ = Impute().execute(data=scaled_data, imputation_method=morpher.config.imputers.DEFAULT)
    
    return imputed_data

data_loader = DataLoader(agg_func_regex, prepare_data_function)

In [37]:
observation_window_ends = [0, 1, 3, 7, 11, 14, 18, 21, 25, 29, 37, 45, 50, 55, 59, 70, 80, 90]

In [38]:
from itertools import product

In [42]:
pipeline_ids = []

In [43]:
for ow_end in observation_window_ends:
    thresholds_numeric = [(x / 100) for x in range(5, 15, 5)]
    observation_windows_numeric = [[x, ow_end] for x in range(-361, ow_end, 30)]
    thresholds_occurring = [0.05]
    observation_windows_occurring = [[-361, ow_end]]

    iterator = product(
        thresholds_numeric,
        observation_windows_numeric,
        thresholds_occurring,
        observation_windows_occurring
    )

    configs = [x for x in iterator]
    targets = [f'cytomegaloviral_disease_onset_from_{ow_end + 1}_days_after_to_365_days_after']
    algorithms = [morpher.config.algorithms.LR]
    samplers = [morpher.config.samplers.RANDOM]
    
    pipeline = training.execute(
        comment=f'NEW baseline-4 cmv post-tx (numeric focus) - {ow_end + 1} to 365d',
        version=f'34.0.0-post-tx-{ow_end + 1}',
        cohort=cohort,
        onset_dataframe=onset_dataframe,
        feature_pipeline=feature_pipeline,
        data_loader=data_loader,
        observation_iterator=configs,
        targets=targets,
        algorithms=algorithms,
        samplers=samplers,
        feature_type_occurring="occurring",
        feature_type_numeric="numeric_binned",
        bin_size=30,
        rfe__run=True,
        rfe__step_size=50
    )
    pipeline_ids.append({'id': pipeline.id, 'comment': pipeline.comment})

Columns (7,23,34,42,54,55,56,57,133,138,146) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (50,122,124,130,182,189,224,245,327,339,443,490,575,595) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (4,18,37,46,49,72,89,96,103,118,139) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (4,19,43,53,56,79,96,103,110,125,141,184,281,282,295,299,324) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7f6907922a20>
Fetching data for Patient (...)


Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


Fitting estimator with 2311 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 236.417ms

Fitting estimator with 2261 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 208.496ms

Fitting estimator with 2211 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 188.143ms

Fitting estimator with 2161 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 202.895ms

Fitting estimator with 2111 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 199.246ms

Fitting estimator with 2061 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 199.584ms

Fitting estimator with 2011 features.
*** Training of model 'LogisticRegression' started.
*** Training of 

Columns (7,23,34,42,54,55,56,57,133,138,146) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (50,122,124,130,182,189,224,245,327,339,443,490,575,595) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (4,18,37,46,49,72,89,96,103,118,139) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (4,19,43,53,56,79,96,103,110,125,141,184,281,282,295,299,324) have mixed types. Specify dtype option on import or set low_memory=False.


...

Fitting estimator with 624 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 204.793ms

Fitting estimator with 574 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 144.743ms

Fitting estimator with 524 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 126.65ms

Fitting estimator with 474 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 146.986ms

Fitting estimator with 424 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 113.977ms

Fitting estimator with 374 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 102.011ms

Fitting estimator with 324 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifi

Columns (6,15,41,64,73,81,86,102,146,223,266,272,281) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (146,147,171,194,298,299,346,356,378,510,512,533,670,761,804,830,832,846,853,855,860,864,925,942,944,959,964) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (4,18,37,83,92,119,141,189,197,205,212,226,301) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (4,19,43,91,100,127,149,197,205,213,284,294,337,387,393,432,440,521) have mixed types. Specify dtype option on import or set low_memory=False.


Fetching data for <fiber.condition.mrns.MRNs object at 0x7f69952eb080>
Fetching data for Patient (...)


Data with input dtype bool, float64 were all converted to float64 by StandardScaler.
Data with input dtype bool, float64 were all converted to float64 by StandardScaler.


Fitting estimator with 460 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 210.085ms

Fitting estimator with 410 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 116.253ms

Fitting estimator with 360 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 129.148ms

Fitting estimator with 310 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 126.019ms

Fitting estimator with 260 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 95.204ms

Fitting estimator with 210 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifier ready. Time elapsed: 86.667ms

Fitting estimator with 160 features.
*** Training of model 'LogisticRegression' started.
*** Training of classifie

In [44]:
from pprint import pprint

In [47]:
pprint(pipeline_ids)

[{'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 1 to 365d',
  'id': 356},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 2 to 365d',
  'id': 357},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 4 to 365d',
  'id': 358},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 8 to 365d',
  'id': 359},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 12 to 365d',
  'id': 360},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 15 to 365d',
  'id': 361},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 19 to 365d',
  'id': 362},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 22 to 365d',
  'id': 363},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 26 to 365d',
  'id': 364},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 30 to 365d',
  'id': 365},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) - 38 to 365d',
  'id': 366},
 {'comment': 'NEW baseline-4 cmv post-tx (numeric focus) -